In [ ]:
# Imports
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import json

In [ ]:
# Functions
def initialise_driver(installed_path, opening_link):
    driver = webdriver.Chrome(installed_path) # specify the filepath where chromedriver is installed
    driver.get(opening_link) # specify a URL for the browser to open
   
    return driver

def get_links(driver):
    get_video_link = driver.find_elements_by_xpath('//*[@id="video-title"]') # find location of links
    links = []
    for i in get_video_link:
        links.append(i.get_attribute('href'))
    
    return links

def initialise_dictionary_list(category, links): # Create a list of dictionaries where each dictionary contains info for one video
    dictionary_list = []
    for i in range(len(links)):
        if links[i] is not None:
            init_dict = {
                "link": links[i],
                "title": "",
                "description": "",
                "views": 0,
                "category": category,
            }

            dictionary_list.append(init_dict)
    
    return dictionary_list

def scrape_info(driver, links, dictionary_list, flag): # Scrape video info -> flag can be "title", "description" or "views"
    wait = WebDriverWait(driver, 180) # selenium specifies a wait time of 180s for the page to finish loading

    if flag is "title":
        for i in range(len(links)):
            if links[i] is not None:
                driver.get(links[i])

                v_title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,"h1.title yt-formatted-string"))).text
                dictionary_list[i]['title'] = v_title

    elif flag is "description":
        for i in range(len(links)):
            if links[i] is not None:
                description = []
                driver.get(links[i])
                v_description_element = wait.until(EC.presence_of_element_located((By.XPATH, 
                "/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[9]/div[2]/ytd-video-secondary-info-renderer/div/ytd-expander/tp-yt-paper-button[2]/yt-formatted-string")))
                ActionChains(driver).move_to_element(v_description_element).click().perform()
                v_full_description = driver.find_elements_by_class_name("style-scope ytd-video-secondary-info-renderer")

                for line in v_full_description:
                    description.append(line.text)
        
                dictionary_list[i]['description'] = description

    elif flag is "views":
        for i in range(len(links)):
            if links[i] is not None:
                driver.get(links[i])
                v_views = wait.until(EC.visibility_of_element_located((By.XPATH, "//*[@id='count']/ytd-video-view-count-renderer/span[1]"))).text
        
                dictionary_list[i]['views'] = v_views

    return dictionary_list

def write_to_json_file(dictionary_list, filename):
    json_string = json.dumps(dictionary_list, indent=4)
    json_file = open(filename, "w")
    json_file.write(json_string)
    json_file.close()